<a href="https://colab.research.google.com/github/dominicwhite/DeepFossil/blob/master/notebooks/01-Simple_FullyConnected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import torch
from torchvision import datasets, transforms
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])
dataset = datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/data/CT/simulated_volumes/128', transform=transform)

In [4]:
dataset.classes

['1', '2', '3', '4', '5']

In [0]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [37]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 16 * 16, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 5)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        print("Preflattened", x.shape)
        x = x.view(-1, 64 * 16 * 16)
        print("Flattened", x.shape)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

from torch import nn, optim

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 5

for e in range(epochs):
    print("Starting epoch", e)
#     running_loss = 0
    for images, labels in dataloader:
        print(images.shape)
        images, labels = images.to('cuda', dtype=torch.float), labels.to('cuda', dtype=torch.float)
        optimizer.zero_grad()
        
        output = model(images)
        print(output)
        break
#         loss = criterion(log_ps, labels)
#         loss.backward()
#         optimizer.step()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=5, bias=True)
  (dropout): Dropout(p=0.25)
)
Starting epoch 0
torch.Size([32, 3, 128, 128])
Preflattened torch.Size([32, 64, 16, 16])
Flattened torch.Size([32, 16384])
tensor([[-0.0198,  0.0069,  0.0014, -0.0042,  0.0225],
        [-0.0270,  0.0190,  0.0019,  0.0068,  0.0221],
        [-0.0212,  0.0158,  0.0081,  0.0181,  0.0157],
        [-0.0215,  0.0158, -0.0004,  0.0096,  0.0246],
        [-0.0342,  0.0048,  0.0004,  0.0021,  0.0109],
        [-0.0252,  0.0189, -0.0027,  0.0027,  0.0161],
        [-0.0292,  0.0236,  0.0082, -0.0004,  0.0162],
        [-0.0197,

In [21]:
images, labels = next(iter(dataloader))
images = images.to('cuda')
model(images)

tensor([[-0.0291, -0.0045, -0.0497, -0.0138,  0.0146],
        [-0.0356, -0.0006, -0.0399, -0.0212,  0.0159],
        [-0.0246,  0.0035, -0.0451, -0.0408,  0.0009],
        ...,
        [-0.0200,  0.0015, -0.0523, -0.0229,  0.0171],
        [-0.0308, -0.0036, -0.0457, -0.0265,  0.0115],
        [-0.0490, -0.0111, -0.0382, -0.0078,  0.0006]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [25]:
images[0][0,:,:]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')